# Equal-Weight S&P 500 Index Fund

_Note: This notebook is part of the course "Algorithmic Trading Using Python - Full Course" by freeCodeCamp.org on YouTube, which was published in December 2020. Since then, the S&P 500 indices have changed slightly and certain pandas functions have been deprecated. This notebook is modified in such a way that (1) we do not go over the AAPL single-stock positioning exercise, (2) API call has been updated according to the changes made by IEX Cloud, (3) we use the pd.concat function to replace the deprecated data_frame.append pandas function, and (4) we skip ahead to Batch API Calls and omit the slow-performing loop over 500 single-calls. Very importantly, the `secret.py` file containing the API token has been changed to `secretapi.py` due to a clash with a Python scripting file, which would not run otherwise._

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course.

We capture the constituents via Wikipedia using the following code.

In [2]:
##Get the latest S&P 500 indices.
import pandas as pd
wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
data = wiki_data[0] # All data is stored in first cell
display(data.head()) # print to check data downloaded
# drop columns 'SEC filings' Since we are dropping columns we have axis = 1 and inplace = True as we are removing these columns in original dataframe
#data.drop(['SEC filings'], axis=1, inplace=True)
sorted_data = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
# Convert the dataframe to csv file
sorted_data.to_csv('SnP500Tickers.csv', mode='w', index=False) #index is False as we don't want to write index in csv file

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


We now import these stocks to our Jupyter Notebook file.

In [3]:
#stocks = pd.read_csv('sp_500_stocks.csv') ##This original file is obsolete.
stocks = pd.read_csv('SnP500Tickers.csv')
stocks
##Note that the Ticker column is now the Symbol column as this is the term Wikipedia uses.

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Health Care Equipment,"Santa Clara, California",2000-06-05,1090872,1999
1,AAL,American Airlines Group,Industrials,Passenger Airlines,"Fort Worth, Texas",2015-03-23,6201,1934
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,1158449,1932
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977
4,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secretapi.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secretapi.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding. Note that the original file is `secret.py`, but the file name is changed for reasons mentioned above.

In [4]:
from secretapi import IEX_CLOUD_API_TOKEN

In [5]:
stocks = pd.read_csv('SnP500Tickers.csv')
stocks

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Health Care Equipment,"Santa Clara, California",2000-06-05,1090872,1999
1,AAL,American Airlines Group,Industrials,Passenger Airlines,"Fort Worth, Texas",2015-03-23,6201,1934
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,1158449,1932
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977
4,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of code.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [7]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [8]:
## Using the pandas.concat function in place of the deprecated "append" function.

symbol_groups = list(chunks(stocks['Symbol'], 100))
#symbol_groups
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

def DF2(symbol):
    return pd.DataFrame([[
        symbol,
        data[symbol]['quote']['latestPrice'],
        data[symbol]['quote']['marketCap'],
        'N/A'
        ]], columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df=DF2(symbol)
        final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)
        
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,117.750,34780502570,N/A
1,AAL,15.040,9819616000,N/A
2,AAP,67.905,4024986099,N/A
3,AAPL,180.880,2845007617760,N/A
4,ABBV,135.450,238973037156,N/A
...,...,...,...,...
498,YUM,133.685,37443428322,N/A
499,ZBH,128.755,26854338290,N/A
500,ZBRA,278.950,14346492785,N/A
501,ZION,29.450,4361565644,N/A


## Calculating the Number of Shares to Buy

We stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [9]:
portfolio_size = input('Enter the value of your portfolio:')

try: val = float(portfolio_size)
##65446599##65 million US dollar##an amount to experiment with##do any amount you like
    
except ValueError:
    print('That is not a number. \nPlease try again with a number.')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:65 million US dollar
That is not a number. 
Please try again with a number.
Enter the value of your portfolio:65446599


In [10]:
position_size = val/len(final_dataframe.index)
for i in range (0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
##the "loc[*,*]" locates the ith row's "Number of Shares to Buy" entry and equate (i.e. change) it to the stated value
##do so for all i in the given range


print('Position Size =', math.floor(position_size))
final_dataframe

Position Size = 130112


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,117.750,34780502570,1104
1,AAL,15.040,9819616000,8651
2,AAP,67.905,4024986099,1916
3,AAPL,180.880,2845007617760,719
4,ABBV,135.450,238973037156,960
...,...,...,...,...
498,YUM,133.685,37443428322,973
499,ZBH,128.755,26854338290,1010
500,ZBRA,278.950,14346492785,466
501,ZION,29.450,4361565644,4418


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [16]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'font_color': font_color,
        'num_format': '$120,000.00',
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'font_color': font_color,
        'num_format': '0',
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

In [18]:
column_formats = {
        'A': ['Ticker', string_format],
        'B': ['Stock Price', dollar_format],
        'C': ['Market Capitalization', dollar_format],
        'D': ['Number of Shares to Buy', integer_format]
    }

for column in column_formats.keys():
    #print(column)
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, string_format)
    ##{column}:{column} for all cells in that column
    #writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1]) ##alternative way
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [19]:
writer.save()

C:\Users\Q52\AppData\Local\Temp\ipykernel_10460\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
